In [ ]:
import kagglehub
import shutil
import os
# Download latest version
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")
print("Path to dataset files:", path)
#copy to current project folder
src = path  
dst = "./dataset"  
os.makedirs(dst, exist_ok=True)
shutil.copytree(src, dst, dirs_exist_ok=True)
print("Copied dataset to:", dst)

c:\Users\35562\.conda\envs\9517project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\35562\.cache\kagglehub\datasets\rupankarmajumdar\crop-pests-dataset\versions\2


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3060 Laptop GPU


In [ ]:
from ultralytics import YOLO
import os
import time
from ultralytics.utils.metrics import ConfusionMatrix

dataset_yaml = os.path.join("dataset", "data.yaml")

print("\n==================== TRAINING YOLO ====================")
model = YOLO("yolov8n.pt")   # small model
model.train(
    data=dataset_yaml,
    epochs=50,
    imgsz=640,
    batch=16,
    name="pest_detector"
)




==================== TRAINING YOLO ====================
Ultralytics 8.3.225  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pest_detector3, nbs=64, nms=False, opset=None, optimize=False, optim

FileNotFoundError: [Errno 2] No such file or directory: 'runs\\detect\\pest_detector\\weights\\best.pt'

In [5]:
print("\n==================== TESTING YOLO ====================")

model = YOLO("runs/detect/pest_detector3/weights/best.pt")

metrics = model.val(data=dataset_yaml, split="test")
print("\n✅ mAP evaluation complete!")


print("\n==== DETECTION METRICS ====")
print("mAP50-95:", metrics.box.map)
print("mAP50:",     metrics.box.map50)
print("Precision:", metrics.box.mp)     
print("Recall:",    metrics.box.mr)     
print("F1 Score:",  metrics.box.f1) 

# Try to get accuracy from YOLO's results_dict (if provided)
if "metrics/accuracy(B)" in metrics.results_dict:
    print("Accuracy:", metrics.results_dict["metrics/accuracy(B)"])
else:
    print("Accuracy: (not directly available in YOLO detection)")

print("\n==================== SAVING PREDICTIONS ====================")

model.predict(
    source="dataset/test/images",
    save=True,
    imgsz=640
)

print("\n✅ Predictions saved at: runs/detect/predict")


###############################################################
# 5. Measure inference time
###############################################################
print("\n==================== INFERENCE TIME ====================")

start = time.time()
model.predict("dataset/test/images", imgsz=640)
end = time.time()

print("Test inference time (seconds):", end - start)

print("\n✅ YOLO workflow completed!")


==================== TESTING YOLO ====================
Ultralytics 8.3.225  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 254.066.1 MB/s, size: 39.9 KB)
val: Scanning E:\onedrive\OneDrive - UNSW\桌面\9517Project\dataset\test\labels.cache... 546 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 546/546  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 7.3it/s 4.8s0.1s
                   all        546        689      0.827      0.729      0.785      0.461
                  Ants         54         87      0.861      0.644      0.743      0.314
                  Bees         40         44      0.907      0.885       0.93      0.442
               Beetles         41         44      0.699      0.705      0.731       0.37
          Caterpillars         46  